# Lake Taupo Lake Levelling Data Maps



In [9]:
import pandas as pd
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import requests
import numpy as np

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import StamenTerrain

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout

%matplotlib inline

In [10]:
#full width notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [11]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

**Error in difference**

In [12]:
#from http://lectureonline.cl.msu.edu/~mmp/labs/error/e2.htm
def differr(e1, e2):
    err = np.sqrt(e1**2 + e2**2)
    return err

error = differr(6,6)

**Widget functions**

In [13]:
def seld1(wd1):
  global date1
  date1 = wd1['new']
    
def seld2(wd2):
  global date2
  date2 = wd2['new']

In [14]:
def go(g):
  global diff, signif
  #dataframes for each date  
  data1 = dfall[dfall['Datetime']==date1]
  data2 = dfall[dfall['Datetime']==date2]
  #difference dataframe, for plotting
  diff = pd.merge(data2, data1, on='siteID')  
  diff['diff'] = (diff['obs_x'] - diff['obs_y']) * 1000

  maxval = max(abs(diff['diff'].min()), abs(diff['diff'].max()))
  vmin = maxval * -1
  vmax = maxval
  
  #plot map
  #imagery
  imagery = StamenTerrain()
  #map  
  ax = plt.axes(projection=imagery.crs)
  ax.set_extent([float(lonmin), float(lonmax), float(latmin), float(latmax)])
  ax.add_image(imagery, 10)

  #observations as coloured points
  plt.scatter(diff['lon_x'], diff['lat_x'], c=diff['diff'], s=100, marker='o', linewidth=0, cmap='seismic', vmin=vmin, vmax=vmax, norm=matplotlib.colors.Normalize(), transform=ccrs.Geodetic())
  
  ##value labels inside symbols
  geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
  text_transform = offset_copy(geodetic_transform, units='dots', x=0, y=0)
  for value, x, y in zip(diff['diff'], diff['lon_x'], diff['lat_x']):
    plt.text(x, y, str(int(round(value))), size=6, color='white', verticalalignment='center', horizontalalignment='center', transform=text_transform)

  #colour bar
  cb = plt.colorbar()
  cb.set_label('Height change (mm)')

  #reference point
  reflat = -38.7148746646
  reflon = 176.082230857
  plt.plot(reflon, reflat, marker='o', markersize=9, color = 'black', linewidth=0, transform=ccrs.Geodetic())
  plt.plot(reflon, reflat, marker='*', markersize=6, color = 'white', linewidth=0, transform=ccrs.Geodetic())
  
  #title
  title = date1+' to '+date2
  plt.title(title)

  #save as file
  image = date1+'-'+date2+'.png'
  plt.savefig(image, dpi=200)

**Data search parameters**

In [15]:
typeID = 'z'
methodID = 'lak'
latmin = '-38.98'
latmax = '-38.63'
lonmin = '175.70'
lonmax = '176.10'

within = 'POLYGON(('+lonmin+'+'+latmax+','+lonmin+'+'+latmin+','+lonmax+'+'+latmin+','+lonmax+'+'+latmax+','+lonmin+'+'+latmax+'))'

**Query**

In [16]:
url = 'https://fits.geonet.org.nz/site'
payload = {'typeID': typeID, 'methodID': methodID, 'within': within}

**Get sites from site query**

In [17]:
r = requests.get(url,params=payload)
jdata = r.json()
features = jdata['features']
dfsite = pd.DataFrame() #empty dataframe
for i, val in enumerate(features):
  geometry = val['geometry']
  lon = geometry['coordinates'][0]
  lat = geometry['coordinates'][1]
  properties = val['properties']
  siteID = properties['siteID']
  height = properties['height']
  name = properties['name']
  #append these to df
  dfsite = dfsite.append({'siteID': siteID, 'lon': lon, 'lat': lat, 'height': height, 'name': name}, ignore_index=True)

**For each siteID, get time-series data and accumulate**

In [18]:
names = ['dt', 'obs', 'err']
dfts = pd.DataFrame()   #empty dataframe

In [19]:
for site in dfsite['siteID']:
  url= 'https://fits.geonet.org.nz/observation?typeID='+typeID+'&siteID='+site+'&methodID='+methodID
  df = pd.read_csv(url, names=names, skiprows=1, parse_dates={"Datetime" : ['dt']})
  df['siteID'] = site
  dfts = dfts.append(df, ignore_index=True)

**Merge the two dataframes on siteID**

In [20]:
dfall = pd.merge(dfsite, dfts, on='siteID')

**Get a list of dates (as strings) for selection with a widget**

In [21]:
dates = dfall['Datetime'].unique()
datestr = dates.astype('datetime64[D]').astype('str').tolist()
datestr.sort()
datestr.append('choose date')
datestr.reverse()

TypeError: Cannot cast DatetimeArray to dtype datetime64[D]

**Date selection section using widgets**

In [ ]:
wd1 = widgets.Select(
    options= datestr,
    description='earlier date:',
    disabled=False
)
wd1.observe(seld1, names='value')

wd2 = widgets.Select(
    options= datestr,
    description='later date:',
    disabled=False
)
wd2.observe(seld2, names='value')

g=widgets.Button(
    description='Plot and make file',
    disabled=False,
    button_style='success',
    tooltip='Click to plot')
g.on_click(go)

widgets.HBox([wd1, wd2, g])

**Look at selected data in different ways**

**Histogram of changes vs siteID**

In [ ]:
dh = diff.plot(kind='bar', color='blue', x='siteID', y='diff', legend=False)
plt.axhline(0, color='black')
plt.axhline(error, color='red', linestyle='--')
plt.axhline(-1*error, color='red', linestyle='--')
dh.set_ylabel('Height change (mm)')

#title
title = date1+' to '+date2
plt.title(title)

#save as file
image = date1+'-'+date2+'_histogram.png'
plt.savefig(image, dpi=200)

**SiteID vs site name, to help users**

In [ ]:
pd.set_option('display.max_columns', 23) #increase column numbers to show all
text = diff[['siteID', 'name_x']].copy().transpose()
text

In [ ]:
#then reset
pd.reset_option('display.max_columns')

**Boxplot of all changes**

In [ ]:
db = diff.boxplot(column='diff', whis=[0,100])
plt.axhline(error, color='red', linestyle='--')
plt.axhline(-1*error, color='red', linestyle='--')
plt.axhline(0, color='red', linestyle='-')
db.set_ylabel('Height change (mm)')

#title
title = date1+' to '+date2+', whisker=min,max'
plt.title(title)

#save as file
image = date1+'-'+date2+'_box.png'
plt.savefig(image, dpi=200)